In [ ]:

nii_folder = '/content/drive/MyDrive/raw data trinity'


In [ ]:
!pip install nibabel fcmeans optuna

ERROR: Could not find a version that satisfies the requirement fcmeans (from versions: none)
ERROR: No matching distribution found for fcmeans


In [ ]:
import pandas as pd

# Load the Excel file
file_path = '/content/drive/MyDrive/phenotypic_csv/classification.xlsx'
data_df = pd.read_excel(file_path)

# Display the first few rows to understand the structure
data_df.head()

,images,DX_GROUP
0,/content/drive/MyDrive/raw data trinity/Trinit...,1
1,/content/drive/MyDrive/raw data trinity/Trinit...,1
2,/content/drive/MyDrive/raw data trinity/Trinit...,1
3,/content/drive/MyDrive/raw data trinity/Trinit...,1
4,/content/drive/MyDrive/raw data trinity/Trinit...,1


In [ ]:
# Replace with actual column names from the output of data_df.columns
image_paths = data_df['images'].values
labels = data_df['DX_GROUP'].map({1: 1, 2: 0}).values

In [ ]:
import nibabel as nib
import cv2
import numpy as np

def preprocess_image(nii_path, target_size=224):
    # Load the .nii image
    nii_img = nib.load(nii_path)
    img_data = nii_img.get_fdata()

    # Select a central slice or mean slice
    slice_2d = img_data[:, :, img_data.shape[2] // 2]

    # Resize to target size
    resized_slice = cv2.resize(slice_2d, (target_size, target_size))

    # Normalize and apply Gaussian smoothing
    normalized_slice = cv2.normalize(resized_slice, None, 0, 255, cv2.NORM_MINMAX)
    blurred_slice = cv2.GaussianBlur(normalized_slice, (5, 5), 0)

    return blurred_slice

# Process all images
images = []
for path in image_paths:
    processed_image = preprocess_image(path)
    images.append(processed_image)

# Convert to numpy arrays
images = np.array(images).reshape(-1, 224, 224, 1)  # Adding channel dimension
labels = np.array(labels)



# Split images and labels into batches of 32




In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input

# Initialize VGG19 model for feature extraction
input_shape = (224, 224, 1)  # The preprocessed image shape with grayscale channel
vgg_base = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Adapt for single-channel input by repeating the grayscale channel three times
input_layer = Input(shape=input_shape)
expanded_input = tf.keras.layers.Concatenate()([input_layer, input_layer, input_layer])  # Expand to 3 channels

# Extract features using VGG19
vgg_features = vgg_base(expanded_input)
feature_extractor = Model(inputs=input_layer, outputs=vgg_features)

# Extract features for the entire dataset
vgg_features_data = feature_extractor.predict(images)

if vgg_features_data is not None:
    for i, features in enumerate(vgg_features_data):
        print(f"Features for image {i}:")
        print(features)
        print("Shape of features:", features.shape)
else:
    print("No features extracted.")

80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 41s 14s/step
Features for image 0:
[[[ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.         27.647371    0.         ...  0.          0.
    0.        ]
  [ 7.0844164  27.870108    0.         ...  0.          0.
    0.        ]
  ...
  [13.571655    0.         13.032555   ...  0.          2.4901075
    0.        ]
  [ 0.          0.          1.1497316  ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]]

 [[ 0.          7.0586004   0.         ...  0.          0.
    0.        ]
  [14.116193   33.29445     0.         ...  0.          0.
    0.        ]
  [24.054806   18.844856    0.         ...  0.          0.
    0.        ]
  ...
  [25.454727    5.3175735  21.36313    ...  0.          0.
    0.        ]
  [ 0.37550557  8.16919    18.454514   ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ... 

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

# Assuming your 'vgg_features_data' and 'labels' are already loaded as NumPy arrays
# Split the data into training and test sets (80% training, 20% testing)
train_features, test_features, train_labels, test_labels = train_test_split(
    vgg_features_data, labels, test_size=0.2, random_state=42
)

# Now, 'test_features' and 'test_labels' are your test data

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam

# Define the model architecture
resnet_model = Sequential([
    Input(shape=train_features.shape[1:]),  # Input shape from training features
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification layer
])

# Compile the model
resnet_model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Set up the ModelCheckpoint callback to save the best model during training
checkpoint = ModelCheckpoint('best_resnet_model.keras', monitor='val_loss', save_best_only=True, mode='min', verbose=1)

# Train the model
history = resnet_model.fit(
    train_features, train_labels,
    epochs=20,
    #batch_size=32,
    validation_split=0.2,
    callbacks=[checkpoint]  # Save the best model based on validation loss
)

Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.3871 - loss: 4.2189
Epoch 1: val_loss improved from inf to 23.85137, saving model to best_resnet_model.keras
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.3871 - loss: 4.2189 - val_accuracy: 0.5000 - val_loss: 23.8514
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 586ms/step - accuracy: 0.5806 - loss: 22.5645
Epoch 2: val_loss improved from 23.85137 to 10.97989, saving model to best_resnet_model.keras
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.5806 - loss: 22.5645 - val_accuracy: 0.5000 - val_loss: 10.9799
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 336ms/step - accuracy: 0.6774 - loss: 6.3869
Epoch 3: val_loss did not improve from 10.97989
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 416ms/step - accuracy: 0.6774 - loss: 6.3869 - val_accuracy: 0.5000 - val_loss: 13.1812
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 352ms/step - accuracy: 0.7419 - loss: 6.7359
Epoch 4: val_loss improved from 10.97989 to 1.48702, saving model to best_resnet_model.keras

In [ ]:
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score, mean_squared_error

# Load the best model saved during training
saved_model = load_model('best_resnet_model.keras')

# Predict on the test set
test_predictions = saved_model.predict(test_features)

# Convert probabilities to binary labels (0 or 1)
predicted_labels = (test_predictions > 0.5).astype(int)

# Calculate test accuracy using sklearn's accuracy_score
test_accuracy = accuracy_score(test_labels, predicted_labels)
print(f"Test Accuracy: {test_accuracy:.4f}")

# Calculate error rate (1 - accuracy)
test_error = 1 - test_accuracy
print(f"Test Error: {test_error:.4f}")

# Optionally, you can also calculate other metrics like MSE (Mean Squared Error)
test_mse = mean_squared_error(test_labels, predicted_labels)
print(f"Test Mean Squared Error: {test_mse:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
Test Accuracy: 0.8000
Test Error: 0.2000
Test Mean Squared Error: 0.2000


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications import ResNet50

# Define the input layer
input_layer = Input(shape=(224, 224, 1))

# Adapt grayscale input to 3 channels
expanded_input = tf.keras.layers.Concatenate()([input_layer, input_layer, input_layer])

# VGG19 for feature extraction
vgg_base = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
vgg_features = vgg_base(expanded_input)

# Flatten VGG19 features
flattened_features = Flatten()(vgg_features)

# ResNet-based classifier
dense1 = Dense(512, activation='relu')(flattened_features)
dropout1 = Dropout(0.5)(dense1)
dense2 = Dense(256, activation='relu')(dropout1)
dropout2 = Dropout(0.5)(dense2)
output_layer = Dense(1, activation='sigmoid')(dropout2)  # Binary classification

# Combine into a hybrid model
hybrid_model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
hybrid_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Save the hybrid model
hybrid_model_save_path = '/content/drive/MyDrive/hybrid_model_vgg19_resnet.h5'
hybrid_model.save(hybrid_model_save_path)

print(f"Hybrid model saved to {hybrid_model_save_path}")

Hybrid model saved to /content/drive/MyDrive/hybrid_model_vgg19_resnet.h5


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
